In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("./drive/MyDrive/BOAZ ADV/")

# Rating

In [3]:
from datetime import datetime
import numpy as np
import glob
import pandas as pd
import re
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import time
import json

In [4]:
platform_list = ["naver_crawling","kakao_crawling","google_crawling"]
naver_list = os.path.join(os.getcwd(),platform_list[0])
os.listdir(naver_list)

['rating_naver_1.csv',
 'image_naver_1.json',
 'review_naver_1.json',
 'user_review_id_naver_1.json',
 'rating_naver_2.csv',
 'review_naver_2.json',
 'user_review_id_naver_2.json',
 'image_naver_2.json',
 'image_naver_3.json',
 'rating_naver_3.csv',
 'review_naver_3.json',
 'user_review_id_naver_3.json']

In [5]:
def to_date(string):
  try:
    x = pd.to_datetime(string,format="%Y.%m.%d").date()
  except:
    x = np.nan
  return x

In [240]:
# rating 
rating = pd.DataFrame()
targetPattern = os.path.join(naver_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp["Platform"] = "Naver"
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]
rating["Timestamp"] = rating["Timestamp"].apply(to_date)
rating.dropna(inplace=True)

In [241]:
kakao_list = os.path.join(os.getcwd(),platform_list[1])
os.listdir(kakao_list)

['kakao_img.json',
 'kakao_review.csv',
 'line5_2160.ipynb',
 'final_kakao_rating.json',
 'review_kakao.json',
 'rating_kakao.csv',
 'bert']

In [ ]:
targetPattern = os.path.join(kakao_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp.dropna(inplace=True)
  temp["Platform"] = "Kakao"
  temp["Timestamp"] = temp["Timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
  temp["Rating"] = temp["Rating"].apply(lambda x : float(x[0]))
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]

In [243]:
my_date = datetime(2021, 4, 30) # 크롤링 날짜로 변경

def change_datetime(input):
    str_list = ['년', '달', '주', '일', '시간']
    p_num = re.compile('[0-9]')
    p_non_num = re.compile('[^0-9]')
    s=""
    n=""
    date_str = input[:-2]

    
    # 숫자 추출
    n_list = p_num.findall(date_str)
    if len(n_list)==1:
        n = int(n_list[0])
    else:
        n_str=""
        for i in n_list:
            n_str = n_str+i
        n = str(n_str)
    
    # 문자 추출
    s_list = p_non_num.findall(date_str)
    if len(s_list)==1:
        s= s_list[0]
    else:
        s_str=""
        for i in s_list:
            s_str = s_str+i
        s = s_str
    n = int(n)
    #  문자에 따라 매핑
    if s == '년':
        tmp_date = my_date + relativedelta(years= (-1)*n)
        new_date = datetime(tmp_date.year, 1, 1) #1월1일 기준
    elif s == '달':
        tmp_date = my_date +  relativedelta(months= (-1)*n)     
        new_date = datetime(tmp_date.year, tmp_date.month, 1) #1일 기준
    elif s == '주':
        new_date = my_date +  relativedelta(days= (-1)*n*7)
    elif s== '일':
        new_date = my_date +  relativedelta(days= (-1)*n)
    elif s=='시간':
        new_date = my_date
    else:
        new_date = my_date
 
    return time.mktime(new_date.timetuple())


In [244]:
google_list = os.path.join(os.getcwd(),platform_list[2])
targetPattern = os.path.join(google_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp["Platform"] = "Google"
  temp['Timestamp'] = temp.apply(lambda x : change_datetime(x['Timestamp']), axis=1)
  temp["Timestamp"] = temp["Timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]
rating.reset_index(drop=True,inplace=True)

# UserID - ReviewID  

In [245]:
# Naver
user_review_json = {}
targetPattern = os.path.join(naver_list,"user_review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    user_review_json.update(temp)

In [246]:
# Kakao
# 아직 미완성

In [247]:
# Google
targetPattern = os.path.join(google_list,"user_review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    user_review_json.update(temp)

# Text Connect

In [248]:
# Naver
review_json = {}
targetPattern = os.path.join(naver_list,"review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    review_json.update(temp)


In [249]:
# Kakao
# 아직 미완성

In [250]:
# Google
targetPattern = os.path.join(google_list,"review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    review_json.update(temp)


# Image Connect

In [251]:
# Naver
image_json = {}
targetPattern = os.path.join(naver_list,"image_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    image_json.update(temp)


In [252]:
# Kakao
# 아직 미완성

In [253]:
# Google
targetPattern = os.path.join(google_list,"image_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    image_json.update(temp)

# Mapping

In [254]:
def user2reviewid(row):
  try:
    user_review = user_review_json.get(row["ItemID"])
    user_review = user_review.get(row["UserID"])
    return user_review
  except:
    return np.nan

def user2review(row):
  try:
    review = review_json.get(row["ItemID"])
    review = review.get(row["ReviewID"])
    return review
  except:
    return "리뷰 없음"

def user2image(row):
  try:
    image = image_json.get(row["ItemID"])
    image = image.get(row["ReviewID"])
    return image
  except:
    return "이미지 없음"

In [255]:
rating["ReviewID"] = rating.apply(lambda x : user2reviewid(x),axis=1)
rating["Review"] = rating.apply(lambda x : user2review(x),axis=1)
rating["Image"] = rating.apply(lambda x : user2image(x),axis=1)
rating.loc[rating["Image"] == "","Image"] = "이미지 없음"
rating.loc[rating["Image"] != "이미지 없음","Image"] = "이미지 있음"
rating = rating[rating["ReviewID"].isnull() == False]

In [274]:
id_filter = rating.groupby("UserID").count()
id_filter = id_filter[id_filter["ItemID"] > 1].index
rating_filter = rating[rating["UserID"].isin(id_filter)]
test = rating_filter.sort_values('Timestamp').groupby('UserID').tail(1)
train = rating_filter[rating_filter.index.isin(test.index) == False]
test = test[test["ItemID"].isin(set(train["ItemID"]))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[test["ItemID"].isin(set(train["ItemID"]))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[test["ItemID"].isin(set(train["ItemID"]))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]

# Cafe 5033개

In [280]:
train.head()

,UserID,ItemID,Rating,Timestamp,Platform,ReviewID,Review,Image
162,5e9d07b68f87a842bc89450b,한티역 노란고무줄,4.5,2021-03-05,Naver,36864316_5e9d07b68f87a842bc89450b,버블티 늘 만족하고 있습니다 꾸벅,이미지 없음
163,5b7b12811c698184496fb5b6,한티역 노란고무줄,3.0,2021-02-20,Naver,36864316_5b7b12811c698184496fb5b6,주말 여알바생이 좀..불친절해요\n와플은 맛나요,이미지 있음
165,5b7b12811c698184496fb5b6,한티역 노란고무줄,3.0,2021-01-17,Naver,36864316_5b7b12811c698184496fb5b6,주말 여알바생이 좀..불친절해요\n와플은 맛나요,이미지 있음
169,5bad907f2f5b2ae7d2c62f7f,한티역 노란고무줄,4.5,2020-12-11,Naver,36864316_5bad907f2f5b2ae7d2c62f7f,조음,이미지 없음
170,5ba06286ff6902d6a9ba1408,한티역 노란고무줄,5.0,2020-12-02,Naver,36864316_5ba06286ff6902d6a9ba1408,완전좋고친절해요,이미지 없음


In [281]:
test.head()

,UserID,ItemID,Rating,Timestamp,Platform,ReviewID,Review,Image
131051,5ebb71498f87a842bc0565e5,광화문역 카페이마,5.0,2000-06-25,Naver,11678597_5ebb71498f87a842bc0565e5,커피도 음료도 정말 맛있어요,이미지 없음
132504,5cece883199881c3785dbe21,광화문역 까페IMA,5.0,2001-09-08,Naver,11678597_5cece883199881c3785dbe21,조아,이미지 없음
215251,102168386001947894894,수색역 투썸플레이스,5.0,2012-01-01,Google,ChZDSUhNMG9nS0VJQ0FnSUNRcWJxVFJBEAE,난 좋던데,이미지 없음
263778,106783083928570207698,건대입구역 간단하지만특별한화피디Cafe,1.0,2012-01-01,Google,ChdDSUhNMG9nS0VJQ0FnSUNnMWZlanF3RRAB,창피함,이미지 없음
215250,110309218145530803640,수색역 투썸플레이스,5.0,2013-01-01,Google,ChZDSUhNMG9nS0VJQ0FnSUNneDYyS0NREAE,이유,이미지 없음


In [277]:
test["ItemID"].nunique()

5033

In [278]:
train["ItemID"].nunique()

5033

In [279]:
len(set(test["ItemID"]) - set(train["ItemID"]))

0